In [ ]:
import pyodbc
import os
import warnings
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import zscore


In [ ]:
df = pd.read_csv('./path/to/noisy/case_control_file/csv')
df

In [ ]:
race_map = pd.read_csv('./race_sire_map.csv',header=None)
race_map.columns = ['ddr_race', 'SIRE_race']
race_map = race_map.dropna()
race_map

In [ ]:
ethnicity_map = pd.read_csv('./ethnicity_sire_map.csv',header=None)
ethnicity_map.columns = ['ddr_eth', 'SIRE_eth']
ethnicity_map

In [ ]:
race_df = pd.merge(df, race_map, how='left', left_on='FirstRace', right_on='ddr_race')
race_df

In [ ]:
df = pd.merge(race_df, ethnicity_map, how='left', left_on='Ethnicity', right_on='ddr_eth')
df

In [ ]:
df['max_bilirubin'] = df['max_bilirubin'].replace(9999999.0, pd.NA)
df['min_bilirubin'] = df['min_bilirubin'].replace(9999999.0, pd.NA)

In [ ]:
#binarize smoking and drop the never assessed or unknown
df = df[~(df['SmokingStatus'] == 'Never Assessed')]
df = df[~(df['SmokingStatus'] == 'Unknown')]
df.SmokingStatus.value_counts()

In [ ]:
conditions = [
    (df['SmokingStatus'] == 'Former'),
    (df['SmokingStatus'] == 'Every Day'),
    (df['SmokingStatus'] == 'Some Days'),
    (df['SmokingStatus'] == 'Light Smoker'),
    (df['SmokingStatus'] == 'Heavy Smoker'),
    (df['SmokingStatus'] == 'Smoker, Current Status Unknown')
    ]

# create a list of the values we want to assign for each condition
values = ['1','1','1','1','1','1']

# create a new column and use np.select to assign values to it using our lists as arguments
df['smoke_hist'] = np.select(conditions, values)

# display updated DataFrame
df.head()

In [ ]:
valid_sex = ['Male', 'Female']
df = df[df['Sex'].isin(valid_sex)]
df.shape

In [ ]:
df.to_csv('./path/to/case_control_file/csv)